# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [1]:
%pylab inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

Populating the interactive namespace from numpy and matplotlib


ModuleNotFoundError: No module named 'sklearn.utils'

In [ ]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print (classes)

In [ ]:
class StrToBytes:
    def __init__(self, fileobj):
        self.fileobj = fileobj
    def read(self, size):
        return self.fileobj.read(size).encode()
    def readline(self, size=-1):
        return self.fileobj.readline(size).encode()

In [ ]:
def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    # YOUR CODE HERE
    
    data_file = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    data_temp = StrToBytes(open(data_file))
    data = pickle.load(data_temp)
    target = [i] * len(data)
    return data, target

In [ ]:
# load all the data
all_data = []
all_target = []

for i in range(len(classes)):
    data, target = load_pose_data(i)
    all_data.extend(data)
    all_target.extend(target)

print ('total number of data', len(all_data))

In [ ]:
# shuffule data
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.7)
training_data = permutation[:n_training_data]

## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [ ]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [ ]:
train_dataset = []
train_targetset = []
for index in permutation:
    train_dataset.append(all_data[index])
    train_targetset.append(all_target[index])
    

clf.fit(train_dataset[:n_training_data], train_targetset[:n_training_data])

In [ ]:
dataset = []
targetset = []
for i in permutation:
    dataset.append(all_data[i])
    targetset.append(all_target[i])
    
print (permutation)
clf.fit(all_data[:n_training_data], all_target[:n_training_data])

### predicting

In [ ]:
clf.predict(all_data[-1:]), all_target[-1:]

In [ ]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

In [ ]:
expected = []
predicted = []
# YOUR CODE HERE
predicted = clf.predict(all_data[:n_training_data])
expected = all_target[:n_training_data]
evaluate(expected, predicted)

## 4. Evaluate on the test data

In [ ]:
expected = []
predicted = []
# YOUR CODE HERE
testData = permutation[n_training_data:]
testData.shape
expected = np.asarray(all_target)[testData]
predicted = clf.predict(np.asarray(all_data)[testData])
evaluate(expected, predicted)

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [ ]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'wb'))

Then, in the application we can load the trained classifier again.

In [ ]:
clf2 = pickle.load(open(ROBOT_POSE_CLF, 'rb'))
clf2.predict(all_data[-1:]), all_target[-1:]

In [ ]:
all_data[-1:]